# How to dynamically update cell content?

The `display()` function allows for updating from an async data source.

In [4]:
var title = display("querying...");

var currentDisplay = display("starting...");

async IAsyncEnumerable<string> Items(){
    for (int i = 0; i < 10; i++){
        yield return $"item {i}";
        await Task.Delay(1000);
    }
}

await foreach (var item in Items()){
    currentDisplay.Update(item);
}

title.Update("done!");


done!

item 9

This applies to any custom display output, for example a `IEnumerable` of content.

So in this case we will render an async source of data over and over again, replacing the table output each time.

In [5]:
var title = display("querying...");

// some custom record
record Widget(string Name, int Value);

// some async source of widgets
async IAsyncEnumerable<Widget> Widgets(){
    for (int i = 0; i < 10; i++){
        yield return new Widget($"widget {i}", i);
        await Task.Delay(1000);
    }
}

var widgets = new List<Widget>();
var currentDisplay = display(widgets);
// display the widgets as they are produced, appending to the table output
await foreach (var widget in Widgets()){
    widgets.Add(widget);
    currentDisplay.Update(widgets); // render all collected widgets so far
}

title.Update("A complete list of widgets:");

A complete list of widgets:

index value 0 Widget { Name = widget 0, Value = 0 } Name widget 0 Value 0 1 Widget { Name = widget 1, Value = 1 } Name widget 1 Value 1 2 Widget { Name = widget 2, Value = 2 } Name widget 2 Value 2 3 Widget { Name = widget 3, Value = 3 } Name widget 3 Value 3 4 Widget { Name = widget 4, Value = 4 } Name widget 4 Value 4 5 Widget { Name = widget 5, Value = 5 } Name widget 5 Value 5 6 Widget { Name = widget 6, Value = 6 } Name widget 6 Value 6 7 Widget { Name = widget 7, Value = 7 } Name widget 7 Value 7 8 Widget { Name = widget 8, Value = 8 } Name widget 8 Value 8 9 Widget { Name = widget 9, Value = 9 } Name widget 9 Value 9

We can also use the `Dataframe` type to produce a better displayed result out of our widgets type.

In [3]:
#r "nuget:Microsoft.Data.Analysis,0.22.2"

using Microsoft.Data.Analysis;

var nameColumn = new StringDataFrameColumn("Name");
var ageColumn = new Int32DataFrameColumn("Age");

// we loop through the data and add it to the columns and redraw the dataframe each time
var title = display("querying...");
var currentDisplay = display(new DataFrame(nameColumn, ageColumn));
await foreach (var widget in Widgets()){
    nameColumn.Append(widget.Name);
    ageColumn.Append(widget.Value);
    currentDisplay.Update(new DataFrame(nameColumn, ageColumn));
}

title.Update("A complete list of widgets:");

Installed Packages Microsoft.Data.Analysis, 0.22.2

Loading extensions from `{UserProfile}\.nuget\packages\microsoft.data.analysis\0.22.2\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

A complete list of widgets:

index,Name,Age
0,widget 0,0
1,widget 1,1
2,widget 2,2
3,widget 3,3
4,widget 4,4
5,widget 5,5
6,widget 6,6
7,widget 7,7
8,widget 8,8
9,widget 9,9
